In [13]:
!jt -t grade3 -T -N -kl

In [1]:
import time
import copy
import queue
import numpy
from math import *
class Node(object):
    def __init__(self, _state):
        self.state = tuple(_state)
        self.parent = None
        self.depth = 0
        self.h = 0
        self.ballsHave = set()
        self.ballsPlaced = set()
    def setParent(self, _Node):
        self.parent = _Node
    def setDepth(self, _depth):
        self.depth = _depth
    def setHeuristic(self, _h):
        self.h = _h
    def setBallsHave(self,parentBalls):
        (self.ballsHave).update(parentBalls)
    def setBallsPlaced(self, parentBallsPlaced):
        (self.ballsPlaced).update(parentBallsPlaced)
    def addBallsHave(self, pickedBall):
        (self.ballsHave).add(pickedBall)
    def removeBallsHave(self, droppedBall):
        (self.ballsHave).remove(droppedBall)
    def addBallsPlaced(self, placedBall):
        (self.ballsPlaced).add(placedBall)
    def getState(self):
        return self.state
    def getParent(self):
        return self.parent
    def getDepth(self):
        return self.depth
    def getHeuristic(self):
        return self.h 
    def getBallsHave(self):
        return self.ballsHave
    def getBallsPlaced(self):
        return self.ballsPlaced
    def printState(self):
        print(self.state)
    def printPath(self,Node):
        while(Node.parent != None):
            Node.printState()
            Node = Node.parent
        Node.printState()

##  <font color='brown'>Part E: A* </font>

As mentioned earlier, A * algorithm is a searching algorithm that searches for the shortest path between the initial and the final state. It is used in various applications, such as maps.
In maps the A* algorithm is used to calculate the shortest distance between the source (initial state) and the destination (final state).
We will use g(n) as the cost that we have given so far and h(n) as the huristic that estimates th cost from node n to the goal state.
we will use f(n) = g(n) + h(n) for this algorithm

h1: manhattan distance from each balls and goal point

h1: Euclidean distance from each balls and goal point

Admissible: Both are Admissible becouse we only calculated the manhattan distances without considering the obsticles, for sure h(n) <= h*(n) 

Admissible: Both are Admissible because we only calculated the manhattan distances without considering the obsticles, for sure h(n) <= h*(n) 

Consistency: h1 is consistent, Because if A is a node and C is the child and G is the goal, 
h(A) <= cost(A,C) + h(c) and this means that values of f(n) along any way is nondecreasing  


In [2]:
def firstHeuristic(node, finalPointNode, ballsPos): #Manhattan
    if(len(ballsPos) == 0):
        return (node.getState()[0]-finalPointNode.getState()[0])+(node.getState()[1]-finalPointNode.getState()[1])
    ballsLeftToBePlaced = []
    distances = []
    sumDist = []
    for ball in ballsPos:
        if(ball not in node.getBallsHave()):
            if(ballsPos[ball] not in node.getBallsPlaced()):
                distances.append(ball)
    for i in range(len(distances)): #DistFromBalls
        sumDist.append(-2*((node.getState()[0]-distances[i][0])+ (node.getState()[1]-distances[i][1])))
    sumDist.append((node.getState()[0]-finalPointNode.getState()[0])+(node.getState()[1]-finalPointNode.getState()[1]))
    return sum(sumDist)

In [3]:
def pathAvailable(node, maze):
    n, m = len(maze), len(maze[0])
    state = node.getState()
    if((state[0] >=0 and state[0] < n) and (state[1] >=0 and state[1] < m)): 
        if(maze[state[0]][state[1]] != '*'): 
            return True
    return False
def goalTest(node, finalPointNode, ballsPos):
    if(len(node.getBallsPlaced()) == len(ballsPos)): #All balls in right place
        if(node.getState() == finalPointNode.getState()): #Reached goalPoint
            return True
    return False

##  <font color='brown'>Part E: First A* </font>

In [4]:
def Astar_1(startPointNode, finalPointNode, bagCapacity, numBalls, ballsPos, maze):
    frontier = []
    frontierSet = set() #ONLY saves hash(ballsPlaced + ballsHave + state)
    explored = set() #ONLY saves hash(ballsPlaced + ballsHave + state)
    actions = [[0,1],[1,0],[0,-1],[-1,0]] # right-down-left-up
    solDistance, stateSeen, stateSeenUnique = 0,0,0
    startPointNode.setHeuristic(firstHeuristic(startPointNode,finalPointNode,ballsPos)+startPointNode.getDepth())
    frontier.append(startPointNode)
    frontierSet.add(hash(str(startPointNode.getBallsPlaced())+str(startPointNode.getBallsHave())+str(startPointNode.getState())))
    if(goalTest(startPointNode, finalPointNode, ballsPos)):
        return 0,1,1
    while(frontier):
        node = frontier.pop() #Every Iteration minimum Heuristic goes to the end of list
#         if(node.getParent()):
#             print("node: ",node.getState()," parent: ",((node.getParent()).getState())," ballsHave:",node.getBallsHave())
        if(hash(str(node.getBallsPlaced())+str(node.getBallsHave())+str(node.getState())) in frontierSet):
            frontierSet.remove(hash(str(node.getBallsPlaced())+str(node.getBallsHave())+str(node.getState())))
        explored.add(hash(str(node.getBallsPlaced())+str(node.getBallsHave())+str(node.getState())))
        if(node.getState() in node.getBallsHave()):
            node.addBallsPlaced(node.getState())
            node.removeBallsHave(node.getState())
        for action in actions:
            child = Node([node.getState()[0]+action[0], node.getState()[1]+action[1]])
            child.setParent(node)
            child.setDepth(node.getDepth()+1)
            child.setBallsHave(node.getBallsHave())
            child.setBallsPlaced(node.getBallsPlaced())
            if((hash(str(child.getBallsPlaced())+str(child.getBallsHave())+str(child.getState())) not in explored) 
               and (hash(str(child.getBallsPlaced())+str(child.getBallsHave())+str(child.getState())) not in frontierSet)): 
                stateSeen = stateSeen+1
                if(pathAvailable(child,maze)):
                   stateSeenUnique = stateSeenUnique+1
                   if(goalTest(child, finalPointNode, ballsPos)):
#                        child.printPath(child)
                       solDistance = child.getDepth()
                       return solDistance, stateSeen, stateSeenUnique
                   child.setHeuristic(firstHeuristic(child,finalPointNode,ballsPos)+child.getDepth())
                   for i in range(len(frontier)):
                       if(len(ballsPos)>5):
                           if((child.getHeuristic()< frontier[i].getHeuristic()) or(i == len(frontier)-1) ):
                               frontier.insert(i,child)
                               frontierSet.add(hash(str(child.getBallsPlaced())+str(child.getBallsHave())+str(child.getState())))
                               break
                       else:
                           if((frontier[i].getHeuristic()<child.getHeuristic()) or (i == len(frontier)-1) ):
                               frontier.insert(i,child)
                               frontierSet.add(hash(str(child.getBallsPlaced())+str(child.getBallsHave())+str(child.getState())))
                               break
                   if(len(frontier) == 0):
                       frontier.append(child)
                       frontierSet.add(hash(str(child.getBallsPlaced())+str(child.getBallsHave())+str(child.getState())))
            ####### Ball handing
            if((action == [-1,0]) and (len(node.getBallsHave())<bagCapacity)):
                if((action == [-1,0]) and (node.getState() in ballsPos) and (node.getState() not in child.getBallsHave()) and (node.getState() not in child.getBallsPlaced())):
                    child_withBall = Node([node.getState()[0],node.getState()[1]])
                    child_withBall.setParent(node)
                    child_withBall.setDepth(node.getDepth()+1)
                    child_withBall.setBallsHave(node.getBallsHave())
                    child_withBall.setBallsPlaced(node.getBallsPlaced())
                    stateSeen = stateSeen+1
                    ballPicked = ballsPos.get(node.getState()) 
                    child_withBall.addBallsHave(ballPicked)
                    child_withBall.setHeuristic(firstHeuristic(child_withBall,finalPointNode,ballsPos)+child_withBall.getDepth())
                    for i in range(len(frontier)):
                       if(len(ballsPos)>5):
                           if((child_withBall.getHeuristic()<frontier[i].getHeuristic()) or(i == len(frontier)-1) ):
                               frontier.insert(i,child_withBall)
                               frontierSet.add(hash(str(child_withBall.getBallsPlaced())+str(child_withBall.getBallsHave())+str(child_withBall.getState())))
                       else:
                           if((frontier[i].getHeuristic()<child_withBall.getHeuristic()) or(i == len(frontier)-1) ):
                               frontier.insert(i,child_withBall)
                               frontierSet.add(hash(str(child_withBall.getBallsPlaced())+str(child_withBall.getBallsHave())+str(child_withBall.getState())))
    return -1, stateSeen, stateSeenUnique

In [5]:
for M in range(1,5):
    print("\n\nmap",M,": ")
    mazeName = str(M)+".txt"
    file = open(mazeName,'r')
    lines = file.readlines()
    count = 0
    countBallsTemp =0
    for line in lines:
        line = line.split()
        if(count ==0):
            n, m = int(line[0]),int(line[1])
            maze = []
            count +=1
        elif(count ==1):
            startPoint = [int(line[0]),int(line[1])]
            count +=1
        elif(count == 2):
            finalPoint = [int(line[0]),int(line[1])]
            count +=1
        elif(count == 3):
            bagCapacity = int(line[0])
            count +=1
        elif(count == 4):
            numBalls = int(line[0])
            countBallsTemp = numBalls
            ballsPos = {}
            count +=1
        elif(count >4 and countBallsTemp>0):
            ballsPos.update({(int(line[0]),int(line[1])):(int(line[2]),int(line[3]))})
            countBallsTemp -=1
            count +=1
        else:
            maze.append(line)
    timeConsumed = []
    for i in range(3):
        startPointNode = Node(startPoint)
        finalPointNode = Node(finalPoint)
        bagCapacity_map = bagCapacity
        numBalls_map = numBalls
        ballsPos_map = ballsPos
        startTime = time.time()
        solDistance, stateSeen, stateSeenUnique = Astar_1(startPointNode, finalPointNode, bagCapacity_map,numBalls_map, ballsPos_map, maze)
        endTime = time.time()
        timeConsumed.append(endTime-startTime)
    print("solDistance",solDistance)
    print("stateSeen",stateSeen)
    print("stateSeenUnique",stateSeenUnique)
    print("time it took:",(endTime-startTime),"sec")
    print("\nAvg time for three operations:",(sum(timeConsumed))/3,"sec")
    print("\n---------------------------------------------------------\n")



map 1 : 
solDistance 20
stateSeen 582
stateSeenUnique 237
time it took: 0.01699042320251465 sec

Avg time for three operations: 0.022986491521199543 sec

---------------------------------------------------------



map 2 : 
solDistance 48
stateSeen 2492
stateSeenUnique 2303
time it took: 0.17889857292175293 sec

Avg time for three operations: 0.1865065097808838 sec

---------------------------------------------------------



map 3 : 
solDistance 68
stateSeen 13632
stateSeenUnique 6674
time it took: 0.9180312156677246 sec

Avg time for three operations: 0.963127056757609 sec

---------------------------------------------------------



map 4 : 
solDistance 92
stateSeen 1259
stateSeenUnique 456
time it took: 0.018988609313964844 sec

Avg time for three operations: 0.020395517349243164 sec

---------------------------------------------------------



##  <font color='brown'>Part E: Second A* </font>

In [6]:
def secondHeuristic(node, finalPointNode, ballsPos): #Euclidean
    if(len(ballsPos) == 0):
        return (node.getState()[0]-finalPointNode.getState()[0]) + (node.getState()[1]-finalPointNode.getState()[1])
    ballsLeftToBePlaced = []
    distances = []
    sumDist = []
    if(len(ballsPos) >5):
        for ball in ballsPos:
            if(ball not in node.getBallsHave()):
                if(ballsPos[ball] not in node.getBallsPlaced()):
                    ballsLeftToBePlaced.append(ball)
        for i in range(len(distances)): #DistFromBalls
            sumDist.append((node.getState()[0]-distances[i][0])**2 + (node.getState()[1]-distances[i][1])**2)
        sumDist.append((node.getState()[0]-finalPointNode.getState()[0])**2 + (node.getState()[1]-finalPointNode.getState()[1])**2)
    return sum(sumDist)

In [7]:
def Astar_2(startPointNode, finalPointNode, bagCapacity, numBalls, ballsPos, maze):
    frontier = []
    frontierSet = set() #ONLY saves hash(ballsPlaced + ballsHave + state)
    explored = set() #ONLY saves hash(ballsPlaced + ballsHave + state)
    actions = [[0,1],[1,0],[0,-1],[-1,0]] # right-down-left-up
    solDistance, stateSeen, stateSeenUnique = 0,0,0
    startPointNode.setHeuristic(secondHeuristic(startPointNode,finalPointNode,ballsPos)+startPointNode.getDepth())
    frontier.append(startPointNode)
    frontierSet.add(hash(str(startPointNode.getBallsPlaced())+str(startPointNode.getBallsHave())+str(startPointNode.getState())))
    if(goalTest(startPointNode, finalPointNode, ballsPos)):
        return 0,1,1
    while(frontier):
        node = frontier.pop() #Every Iteration minimum Heuristic goes to the end of list
#         if(node.getParent()):
#             print("node: ",node.getState()," parent: ",((node.getParent()).getState())," ballsHave:",node.getBallsHave())
        if(hash(str(node.getBallsPlaced())+str(node.getBallsHave())+str(node.getState())) in frontierSet):
            frontierSet.remove(hash(str(node.getBallsPlaced())+str(node.getBallsHave())+str(node.getState())))
        explored.add(hash(str(node.getBallsPlaced())+str(node.getBallsHave())+str(node.getState())))
        if(node.getState() in node.getBallsHave()):
            node.addBallsPlaced(node.getState())
            node.removeBallsHave(node.getState())
        for action in actions:
            child = Node([node.getState()[0]+action[0], node.getState()[1]+action[1]])
            child.setParent(node)
            child.setDepth(node.getDepth()+1)
            child.setBallsHave(node.getBallsHave())
            child.setBallsPlaced(node.getBallsPlaced())
            if((hash(str(child.getBallsPlaced())+str(child.getBallsHave())+str(child.getState())) not in explored) 
               and (hash(str(child.getBallsPlaced())+str(child.getBallsHave())+str(child.getState())) not in frontierSet)): 
                stateSeen = stateSeen+1
                if(pathAvailable(child,maze)):
                   stateSeenUnique = stateSeenUnique+1
                   if(goalTest(child, finalPointNode, ballsPos)):
#                        child.printPath(child)
                       solDistance = child.getDepth()
                       return solDistance, stateSeen, stateSeenUnique
                   child.setHeuristic(secondHeuristic(child,finalPointNode,ballsPos)+child.getDepth())
                   for i in range(len(frontier)):
                       if(len(ballsPos)>5):
                           if((child.getHeuristic()< frontier[i].getHeuristic()) or(i == len(frontier)-1) ):
                               frontier.insert(i,child)
                               frontierSet.add(hash(str(child.getBallsPlaced())+str(child.getBallsHave())+str(child.getState())))
                               break
                       else:
                           if((frontier[i].getHeuristic()<child.getHeuristic()) or (i == len(frontier)-1) ):
                               frontier.insert(i,child)
                               frontierSet.add(hash(str(child.getBallsPlaced())+str(child.getBallsHave())+str(child.getState())))
                               break
                   if(len(frontier) == 0):
                       frontier.append(child)
                       frontierSet.add(hash(str(child.getBallsPlaced())+str(child.getBallsHave())+str(child.getState())))
            ####### Ball handing
            if((action == [-1,0]) and (len(node.getBallsHave())<bagCapacity)):
                if((action == [-1,0]) and (node.getState() in ballsPos) and (node.getState() not in child.getBallsHave()) and (node.getState() not in child.getBallsPlaced())):
                    child_withBall = Node([node.getState()[0],node.getState()[1]])
                    child_withBall.setParent(node)
                    child_withBall.setDepth(node.getDepth()+1)
                    child_withBall.setBallsHave(node.getBallsHave())
                    child_withBall.setBallsPlaced(node.getBallsPlaced())
                    stateSeen = stateSeen+1
                    ballPicked = ballsPos.get(node.getState()) 
                    child_withBall.addBallsHave(ballPicked)
                    child_withBall.setHeuristic(secondHeuristic(child_withBall,finalPointNode,ballsPos)+child_withBall.getDepth())
                    for i in range(len(frontier)):
                       if(len(ballsPos)>5):
                           if((child_withBall.getHeuristic()<frontier[i].getHeuristic()) or(i == len(frontier)-1) ):
                               frontier.insert(i,child_withBall)
                               frontierSet.add(hash(str(child_withBall.getBallsPlaced())+str(child_withBall.getBallsHave())+str(child_withBall.getState())))
                       else:
                           if((frontier[i].getHeuristic()<child_withBall.getHeuristic()) or(i == len(frontier)-1) ):
                               frontier.insert(i,child_withBall)
                               frontierSet.add(hash(str(child_withBall.getBallsPlaced())+str(child_withBall.getBallsHave())+str(child_withBall.getState())))
    return -1, stateSeen, stateSeenUnique

In [8]:
for M in range(1,5):
    print("\n\nmap",M,": ")
    mazeName = str(M)+".txt"
    file = open(mazeName,'r')
    lines = file.readlines()
    count = 0
    countBallsTemp =0
    for line in lines:
        line = line.split()
        if(count ==0):
            n, m = int(line[0]),int(line[1])
            maze = []
            count +=1
        elif(count ==1):
            startPoint = [int(line[0]),int(line[1])]
            count +=1
        elif(count == 2):
            finalPoint = [int(line[0]),int(line[1])]
            count +=1
        elif(count == 3):
            bagCapacity = int(line[0])
            count +=1
        elif(count == 4):
            numBalls = int(line[0])
            countBallsTemp = numBalls
            ballsPos = {}
            count +=1
        elif(count >4 and countBallsTemp>0):
            ballsPos.update({(int(line[0]),int(line[1])):(int(line[2]),int(line[3]))})
            countBallsTemp -=1
            count +=1
        else:
            maze.append(line)
    timeConsumed = []
    for i in range(3):
        startPointNode = Node(startPoint)
        finalPointNode = Node(finalPoint)
        bagCapacity_map = bagCapacity
        numBalls_map = numBalls
        ballsPos_map = ballsPos
        startTime = time.time()
        solDistance, stateSeen, stateSeenUnique = Astar_2(startPointNode, finalPointNode, bagCapacity_map,numBalls_map, ballsPos_map, maze)
        endTime = time.time()
        timeConsumed.append(endTime-startTime)
    print("solDistance",solDistance)
    print("stateSeen",stateSeen)
    print("stateSeenUnique",stateSeenUnique)
    print("time it took:",(endTime-startTime),"sec")
    print("\nAvg time for three operations:",(sum(timeConsumed))/3,"sec")
    print("\n---------------------------------------------------------\n")



map 1 : 
solDistance 20
stateSeen 1106
stateSeenUnique 304
time it took: 0.030982494354248047 sec

Avg time for three operations: 0.03664541244506836 sec

---------------------------------------------------------



map 2 : 
solDistance 48
stateSeen 2492
stateSeenUnique 2303
time it took: 0.16890287399291992 sec

Avg time for three operations: 0.1752324104309082 sec

---------------------------------------------------------



map 3 : 
solDistance 68
stateSeen 59886
stateSeenUnique 29470
time it took: 10.165885210037231 sec

Avg time for three operations: 10.153114875157675 sec

---------------------------------------------------------



map 4 : 
solDistance 92
stateSeen 1259
stateSeenUnique 456
time it took: 0.018973350524902344 sec

Avg time for three operations: 0.019307374954223633 sec

---------------------------------------------------------



##  <font color='brown'>Part F: First Weighted A* with alpha= 1.5, 3  </font>

In [9]:
def Astar_1_weighted(startPointNode, finalPointNode, bagCapacity, numBalls, ballsPos, maze,alpha):
    frontier = []
    frontierSet = set() #ONLY saves hash(ballsPlaced + ballsHave + state)
    explored = set() #ONLY saves hash(ballsPlaced + ballsHave + state)
    actions = [[0,1],[1,0],[0,-1],[-1,0]] # right-down-left-up
    solDistance, stateSeen, stateSeenUnique = 0,0,0
    startPointNode.setHeuristic(alpha*firstHeuristic(startPointNode,finalPointNode,ballsPos)+startPointNode.getDepth())
    frontier.append(startPointNode)
    frontierSet.add(hash(str(startPointNode.getBallsPlaced())+str(startPointNode.getBallsHave())+str(startPointNode.getState())))
    if(goalTest(startPointNode, finalPointNode, ballsPos)):
        return 0,1,1
    while(frontier):
        node = frontier.pop() #Every Iteration minimum Heuristic goes to the end of list
#         if(node.getParent()):
#             print("node: ",node.getState()," parent: ",((node.getParent()).getState())," ballsHave:",node.getBallsHave())
        if(hash(str(node.getBallsPlaced())+str(node.getBallsHave())+str(node.getState())) in frontierSet):
            frontierSet.remove(hash(str(node.getBallsPlaced())+str(node.getBallsHave())+str(node.getState())))
        explored.add(hash(str(node.getBallsPlaced())+str(node.getBallsHave())+str(node.getState())))
        if(node.getState() in node.getBallsHave()):
            node.addBallsPlaced(node.getState())
            node.removeBallsHave(node.getState())
        for action in actions:
            child = Node([node.getState()[0]+action[0], node.getState()[1]+action[1]])
            child.setParent(node)
            child.setDepth(node.getDepth()+1)
            child.setBallsHave(node.getBallsHave())
            child.setBallsPlaced(node.getBallsPlaced())
            if((hash(str(child.getBallsPlaced())+str(child.getBallsHave())+str(child.getState())) not in explored) 
               and (hash(str(child.getBallsPlaced())+str(child.getBallsHave())+str(child.getState())) not in frontierSet)): 
                stateSeen = stateSeen+1
                if(pathAvailable(child,maze)):
                   stateSeenUnique = stateSeenUnique+1
                   if(goalTest(child, finalPointNode, ballsPos)):
#                        child.printPath(child)
                       solDistance = child.getDepth()
                       return solDistance, stateSeen, stateSeenUnique
                   child.setHeuristic(alpha*firstHeuristic(child,finalPointNode,ballsPos)+child.getDepth())
                   for i in range(len(frontier)):
                       if(len(ballsPos)>5):
                           if((child.getHeuristic()< frontier[i].getHeuristic()) or(i == len(frontier)-1) ):
                               frontier.insert(i,child)
                               frontierSet.add(hash(str(child.getBallsPlaced())+str(child.getBallsHave())+str(child.getState())))
                               break
                       else:
                           if((frontier[i].getHeuristic()<child.getHeuristic()) or (i == len(frontier)-1) ):
                               frontier.insert(i,child)
                               frontierSet.add(hash(str(child.getBallsPlaced())+str(child.getBallsHave())+str(child.getState())))
                               break
                   if(len(frontier) == 0):
                       frontier.append(child)
                       frontierSet.add(hash(str(child.getBallsPlaced())+str(child.getBallsHave())+str(child.getState())))
            ####### Ball handing
            if((action == [-1,0]) and (len(node.getBallsHave())<bagCapacity)):
                if((action == [-1,0]) and (node.getState() in ballsPos) and (node.getState() not in child.getBallsHave()) and (node.getState() not in child.getBallsPlaced())):
                    child_withBall = Node([node.getState()[0],node.getState()[1]])
                    child_withBall.setParent(node)
                    child_withBall.setDepth(node.getDepth()+1)
                    child_withBall.setBallsHave(node.getBallsHave())
                    child_withBall.setBallsPlaced(node.getBallsPlaced())
                    stateSeen = stateSeen+1
                    ballPicked = ballsPos.get(node.getState()) 
                    child_withBall.addBallsHave(ballPicked)
                    child_withBall.setHeuristic(alpha*firstHeuristic(child_withBall,finalPointNode,ballsPos)+child_withBall.getDepth())
                    for i in range(len(frontier)):
                       if(len(ballsPos)>5):
                           if((child_withBall.getHeuristic()<frontier[i].getHeuristic()) or(i == len(frontier)-1) ):
                               frontier.insert(i,child_withBall)
                               frontierSet.add(hash(str(child_withBall.getBallsPlaced())+str(child_withBall.getBallsHave())+str(child_withBall.getState())))
                       else:
                           if((frontier[i].getHeuristic()<child_withBall.getHeuristic()) or(i == len(frontier)-1) ):
                               frontier.insert(i,child_withBall)
                               frontierSet.add(hash(str(child_withBall.getBallsPlaced())+str(child_withBall.getBallsHave())+str(child_withBall.getState())))
    return -1, stateSeen, stateSeenUnique

In [10]:
for M in range(1,5):
    print("\n\nmap",M,": \n")
    mazeName = str(M)+".txt"
    file = open(mazeName,'r')
    lines = file.readlines()
    count = 0
    countBallsTemp =0
    for line in lines:
        line = line.split()
        if(count ==0):
            n, m = int(line[0]),int(line[1])
            maze = []
            count +=1
        elif(count ==1):
            startPoint = [int(line[0]),int(line[1])]
            count +=1
        elif(count == 2):
            finalPoint = [int(line[0]),int(line[1])]
            count +=1
        elif(count == 3):
            bagCapacity = int(line[0])
            count +=1
        elif(count == 4):
            numBalls = int(line[0])
            countBallsTemp = numBalls
            ballsPos = {}
            count +=1
        elif(count >4 and countBallsTemp>0):
            ballsPos.update({(int(line[0]),int(line[1])):(int(line[2]),int(line[3]))})
            countBallsTemp -=1
            count +=1
        else:
            maze.append(line)
    timeConsumed = []
    print('alpha = ',1.5)
    for i in range(3):
        startPointNode = Node(startPoint)
        finalPointNode = Node(finalPoint)
        bagCapacity_map = bagCapacity
        numBalls_map = numBalls
        ballsPos_map = ballsPos
        startTime = time.time()
        solDistance, stateSeen, stateSeenUnique = Astar_1_weighted(startPointNode, finalPointNode, bagCapacity_map,numBalls_map, ballsPos_map, maze,1.5)
        endTime = time.time()
        timeConsumed.append(endTime-startTime)
    print("solDistance",solDistance)
    print("stateSeen",stateSeen)
    print("stateSeenUnique",stateSeenUnique)
    print("time it took:",(endTime-startTime),"sec")
    print("\nAvg time for three operations:",(sum(timeConsumed))/3,"sec")
    print("\n---------------------------------------------------------\n")
    print('alpha = ',3)
    for i in range(3):
        startPointNode = Node(startPoint)
        finalPointNode = Node(finalPoint)
        bagCapacity_map = bagCapacity
        numBalls_map = numBalls
        ballsPos_map = ballsPos
        startTime = time.time()
        solDistance, stateSeen, stateSeenUnique = Astar_1_weighted(startPointNode, finalPointNode, bagCapacity_map,numBalls_map, ballsPos_map, maze,3)
        endTime = time.time()
        timeConsumed.append(endTime-startTime)
    print("solDistance",solDistance)
    print("stateSeen",stateSeen)
    print("stateSeenUnique",stateSeenUnique)
    print("time it took:",(endTime-startTime),"sec")
    print("\nAvg time for three operations:",(sum(timeConsumed))/3,"sec")
    print("\n---------------------------------------------------------\n")
    print("\n---------------------------------------------------------\n")
    print("\n---------------------------------------------------------\n")



map 1 : 

alpha =  1.5
solDistance 20
stateSeen 574
stateSeenUnique 229
time it took: 0.023986101150512695 sec

Avg time for three operations: 0.019655148188273113 sec

---------------------------------------------------------

alpha =  3
solDistance 20
stateSeen 677
stateSeenUnique 275
time it took: 0.030979633331298828 sec

Avg time for three operations: 0.04397424062093099 sec

---------------------------------------------------------


---------------------------------------------------------


---------------------------------------------------------



map 2 : 

alpha =  1.5
solDistance 48
stateSeen 2492
stateSeenUnique 2303
time it took: 0.11686038970947266 sec

Avg time for three operations: 0.13799381256103516 sec

---------------------------------------------------------

alpha =  3
solDistance 48
stateSeen 2492
stateSeenUnique 2303
time it took: 0.1271834373474121 sec

Avg time for three operations: 0.26909899711608887 sec

-------------------------------------------------

##  <font color='brown'>Part F: Second Weighted A* with alpha= 1.5,3 </font>

In [11]:
def Astar_2_weighted(startPointNode, finalPointNode, bagCapacity, numBalls, ballsPos, maze, alpha):
    frontier = []
    frontierSet = set() #ONLY saves hash(ballsPlaced + ballsHave + state)
    explored = set() #ONLY saves hash(ballsPlaced + ballsHave + state)
    actions = [[0,1],[1,0],[0,-1],[-1,0]] # right-down-left-up
    solDistance, stateSeen, stateSeenUnique = 0,0,0
    startPointNode.setHeuristic(alpha*secondHeuristic(startPointNode,finalPointNode,ballsPos)+startPointNode.getDepth())
    frontier.append(startPointNode)
    frontierSet.add(hash(str(startPointNode.getBallsPlaced())+str(startPointNode.getBallsHave())+str(startPointNode.getState())))
    if(goalTest(startPointNode, finalPointNode, ballsPos)):
        return 0,1,1
    while(frontier):
        node = frontier.pop() #Every Iteration minimum Heuristic goes to the end of list
#         if(node.getParent()):
#             print("node: ",node.getState()," parent: ",((node.getParent()).getState())," ballsHave:",node.getBallsHave())
        if(hash(str(node.getBallsPlaced())+str(node.getBallsHave())+str(node.getState())) in frontierSet):
            frontierSet.remove(hash(str(node.getBallsPlaced())+str(node.getBallsHave())+str(node.getState())))
        explored.add(hash(str(node.getBallsPlaced())+str(node.getBallsHave())+str(node.getState())))
        if(node.getState() in node.getBallsHave()):
            node.addBallsPlaced(node.getState())
            node.removeBallsHave(node.getState())
        for action in actions:
            child = Node([node.getState()[0]+action[0], node.getState()[1]+action[1]])
            child.setParent(node)
            child.setDepth(node.getDepth()+1)
            child.setBallsHave(node.getBallsHave())
            child.setBallsPlaced(node.getBallsPlaced())
            if((hash(str(child.getBallsPlaced())+str(child.getBallsHave())+str(child.getState())) not in explored) 
               and (hash(str(child.getBallsPlaced())+str(child.getBallsHave())+str(child.getState())) not in frontierSet)): 
                stateSeen = stateSeen+1
                if(pathAvailable(child,maze)):
                   stateSeenUnique = stateSeenUnique+1
                   if(goalTest(child, finalPointNode, ballsPos)):
#                        child.printPath(child)
                       solDistance = child.getDepth()
                       return solDistance, stateSeen, stateSeenUnique
                   child.setHeuristic(alpha*secondHeuristic(child,finalPointNode,ballsPos)+child.getDepth())
                   for i in range(len(frontier)):
                       if(len(ballsPos)>5):
                           if((child.getHeuristic()< frontier[i].getHeuristic()) or(i == len(frontier)-1) ):
                               frontier.insert(i,child)
                               frontierSet.add(hash(str(child.getBallsPlaced())+str(child.getBallsHave())+str(child.getState())))
                               break
                       else:
                           if((frontier[i].getHeuristic()<child.getHeuristic()) or (i == len(frontier)-1) ):
                               frontier.insert(i,child)
                               frontierSet.add(hash(str(child.getBallsPlaced())+str(child.getBallsHave())+str(child.getState())))
                               break
                   if(len(frontier) == 0):
                       frontier.append(child)
                       frontierSet.add(hash(str(child.getBallsPlaced())+str(child.getBallsHave())+str(child.getState())))
            ####### Ball handing
            if((action == [-1,0]) and (len(node.getBallsHave())<bagCapacity)):
                if((action == [-1,0]) and (node.getState() in ballsPos) and (node.getState() not in child.getBallsHave()) and (node.getState() not in child.getBallsPlaced())):
                    child_withBall = Node([node.getState()[0],node.getState()[1]])
                    child_withBall.setParent(node)
                    child_withBall.setDepth(node.getDepth()+1)
                    child_withBall.setBallsHave(node.getBallsHave())
                    child_withBall.setBallsPlaced(node.getBallsPlaced())
                    stateSeen = stateSeen+1
                    ballPicked = ballsPos.get(node.getState()) 
                    child_withBall.addBallsHave(ballPicked)
                    child_withBall.setHeuristic(alpha*secondHeuristic(child_withBall,finalPointNode,ballsPos)+child_withBall.getDepth())
                    for i in range(len(frontier)):
                       if(len(ballsPos)>5):
                           if((child_withBall.getHeuristic()<frontier[i].getHeuristic()) or(i == len(frontier)-1) ):
                               frontier.insert(i,child_withBall)
                               frontierSet.add(hash(str(child_withBall.getBallsPlaced())+str(child_withBall.getBallsHave())+str(child_withBall.getState())))
                       else:
                           if((frontier[i].getHeuristic()<child_withBall.getHeuristic()) or(i == len(frontier)-1) ):
                               frontier.insert(i,child_withBall)
                               frontierSet.add(hash(str(child_withBall.getBallsPlaced())+str(child_withBall.getBallsHave())+str(child_withBall.getState())))
    return -1, stateSeen, stateSeenUnique

In [12]:
for M in range(1,5):
    print("\n\nmap",M,": \n")
    mazeName = str(M)+".txt"
    file = open(mazeName,'r')
    lines = file.readlines()
    count = 0
    countBallsTemp =0
    for line in lines:
        line = line.split()
        if(count ==0):
            n, m = int(line[0]),int(line[1])
            maze = []
            count +=1
        elif(count ==1):
            startPoint = [int(line[0]),int(line[1])]
            count +=1
        elif(count == 2):
            finalPoint = [int(line[0]),int(line[1])]
            count +=1
        elif(count == 3):
            bagCapacity = int(line[0])
            count +=1
        elif(count == 4):
            numBalls = int(line[0])
            countBallsTemp = numBalls
            ballsPos = {}
            count +=1
        elif(count >4 and countBallsTemp>0):
            ballsPos.update({(int(line[0]),int(line[1])):(int(line[2]),int(line[3]))})
            countBallsTemp -=1
            count +=1
        else:
            maze.append(line)
    timeConsumed = []
    print('alpha = ',0.75)
    for i in range(3):
        startPointNode = Node(startPoint)
        finalPointNode = Node(finalPoint)
        bagCapacity_map = bagCapacity
        numBalls_map = numBalls
        ballsPos_map = ballsPos
        startTime = time.time()
        solDistance, stateSeen, stateSeenUnique = Astar_2_weighted(startPointNode, finalPointNode, bagCapacity_map,numBalls_map, ballsPos_map, maze,0.75)
        endTime = time.time()
        timeConsumed.append(endTime-startTime)
    print("solDistance",solDistance)
    print("stateSeen",stateSeen)
    print("stateSeenUnique",stateSeenUnique)
    print("time it took:",(endTime-startTime),"sec")
    print("\nAvg time for three operations:",(sum(timeConsumed))/3,"sec")
    print("\n---------------------------------------------------------\n")
    print('alpha = ',0.5)
    for i in range(3):
        startPointNode = Node(startPoint)
        finalPointNode = Node(finalPoint)
        bagCapacity_map = bagCapacity
        numBalls_map = numBalls
        ballsPos_map = ballsPos
        startTime = time.time()
        solDistance, stateSeen, stateSeenUnique = Astar_2_weighted(startPointNode, finalPointNode, bagCapacity_map,numBalls_map, ballsPos_map, maze,0.5)
        endTime = time.time()
        timeConsumed.append(endTime-startTime)
    print("solDistance",solDistance)
    print("stateSeen",stateSeen)
    print("stateSeenUnique",stateSeenUnique)
    print("time it took:",(endTime-startTime),"sec")
    print("\nAvg time for three operations:",(sum(timeConsumed))/3,"sec")
    print("\n---------------------------------------------------------\n")
    print("\n---------------------------------------------------------\n")
    print("\n---------------------------------------------------------\n")



map 1 : 

alpha =  0.75
solDistance 20
stateSeen 1106
stateSeenUnique 304
time it took: 0.03799295425415039 sec

Avg time for three operations: 0.037310123443603516 sec

---------------------------------------------------------

alpha =  0.5
solDistance 20
stateSeen 1106
stateSeenUnique 304
time it took: 0.028983592987060547 sec

Avg time for three operations: 0.07029112180074056 sec

---------------------------------------------------------


---------------------------------------------------------


---------------------------------------------------------



map 2 : 

alpha =  0.75
solDistance 48
stateSeen 2492
stateSeenUnique 2303
time it took: 0.13092589378356934 sec

Avg time for three operations: 0.11760552724202473 sec

---------------------------------------------------------

alpha =  0.5
solDistance 48
stateSeen 2492
stateSeenUnique 2303
time it took: 0.1159200668334961 sec

Avg time for three operations: 0.2376080354054769 sec

-------------------------------------------